### Import libraries etc.

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

Most of what's below is from a previous notebook, the nnls attempt is near the bottom.

In [ ]:
df = pd.read_csv("CSD_output_formulae_50k.csv")

In [ ]:
def molparsed(row):
    baseformula = {"H":0,"He":0,"Li":0,"Be":0,"B":0,"C":0,"N":0,"O":0,"F":0,"Ne":0,
    "Na":0,"Mg":0,"Al":0,"Si":0,"P":0,"S":0,"Cl":0,"Ar":0,"K":0,"Ca":0,
    "Sc":0,"Ti":0,"V":0,"Cr":0,"Mn":0,"Fe":0,"Co":0,"Ni":0,"Cu":0,"Zn":0,
    "Ga":0,"Ge":0,"As":0,"Se":0,"Br":0,"Kr":0,"Rb":0,"Sr":0,"Y":0,"Zr":0,
    "Nb":0,"Mo":0,"Tc":0,"Ru":0,"Rh":0,"Pd":0,"Ag":0,"Cd":0,"In":0,"Sn":0,
    "Sb":0,"Te":0,"I":0,"Xe":0,"Cs":0,"Ba":0,"La":0,"Ce":0,"Pr":0,"Nd":0,
    "Pm":0,"Sm":0,"Eu":0,"Gd":0,"Tb":0,"Dy":0,"Ho":0,"Er":0,"Tm":0,"Yb":0,
    "Lu":0,"Hf":0,"Ta":0,"W":0,"Re":0,"Os":0,"Ir":0,"Pt":0,"Au":0,"Hg":0,
    "Tl":0,"Pb":0,"Bi":0,"Po":0,"At":0,"Rn":0,"Fr":0,"Ra":0,"Ac":0,"Th":0,
    "Pa":0,"U":0,"Np":0,"Pu":0,"Am":0,"Cm":0,"Bk":0,"Cf":0,"Es":0,"Fm":0,
    "Md":0,"No":0,"Lr":0,"Rf":0,"Db":0,"Sg":0,"Bh":0,"Hs":0,"Mt":0}
    formula = row["formula"]
    Z = row["Z"]
    if formula[-1] == "n":
            return 0
            
    parsedform = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    for element in parsedform:
        baseformula[element[0]] = float(element[1])*Z
        #print element
    return baseformula

In [ ]:
df["UnitFormula"] = df.apply(molparsed,axis=1)
df = df[df.UnitFormula !=0]
print df["UnitFormula"]

In [ ]:
X = []
label = []
i = 0
for x in df["UnitFormula"]:
    temp = []
    for key, value in x.iteritems():
        temp.append(value)
        if i == 0:
            label.append(key)            
    X.append(temp)
    i+=1
y = df["Volume"]

In [ ]:
#print X,label
X = np.matrix(X) # changing X array to a numpy matrix
y = np.array(y) # changing y dataframe column to numpy array
print y

As the nnls function works with arrays/matrices the inputs had to be converted.

In [ ]:
df50 = pd.DataFrame(data=X, columns=label)

In [ ]:
from sklearn import linear_model, svm
reg = linear_model.LinearRegression()
reg.fit(X,y)
for this in zip(label,reg.coef_):
    print this

In [ ]:
from scipy.optimize import nnls 
x, rnorm = nnls(X,y)
print zip(label,x), rnorm

As the previous results (can be executed a couple cells above) were riddled with negative results non-negative least squares (nnls) was attempted and the results proved to be slightly better however there were still some extremely errenous results (>100), such as Zn (404.74) or Ca (337.29)